In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [43]:
df = pd.read_csv('diabetes.csv')

In [44]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [45]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [46]:
x = df.iloc[:,0:-1]
y = df.iloc[:,-1]

In [47]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)

In [48]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_trf = scaler.fit_transform(x_train)
x_test_trf = scaler.transform(x_test)

In [49]:
print(x_train_trf.shape)
print(x_test_trf.shape)

(614, 8)
(154, 8)


In [50]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [51]:
model = Sequential()

model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [52]:
model.fit(batch_size=32,x = x_train_trf,y = y_train,epochs=100,validation_data=(x_test_trf,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - accuracy: 0.3397 - loss: 0.7719 - val_accuracy: 0.5065 - val_loss: 0.6945
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5574 - loss: 0.6914 - val_accuracy: 0.7403 - val_loss: 0.6368
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6696 - loss: 0.6529 - val_accuracy: 0.7727 - val_loss: 0.5946
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7449 - loss: 0.5968 - val_accuracy: 0.7597 - val_loss: 0.5606
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7553 - loss: 0.5649 - val_accuracy: 0.7727 - val_loss: 0.5331
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7479 - loss: 0.5383 - val_accuracy: 0.7987 - val_loss: 0.5138
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7619 - loss: 0.5218 - val_accuracy: 0.7922 - val_loss: 0.4978
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7611 - loss: 0.5131 - val_accuracy: 0.7857 - 

In [53]:
import keras_tuner as kt

In [54]:
def build_model(hp):

  model = Sequential()

  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optims = hp.Choice('optimizer',values = ['sgd','adam','rmsprop','adadelta'])

  model.compile(optimizer=optims,loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [55]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [56]:
tuner.search(x_train_trf,y_train,epochs=5,validation_data=(x_test_trf,y_test))

In [57]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [58]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [59]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [60]:
model.fit(x_train_trf,y_train,epochs=100,initial_epoch=6,batch_size=32,validation_data=(x_test_trf,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 97ms/step - accuracy: 0.7053 - loss: 0.5668 - val_accuracy: 0.7727 - val_loss: 0.5185
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7608 - loss: 0.5287 - val_accuracy: 0.7857 - val_loss: 0.5028
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7535 - loss: 0.5271 - val_accuracy: 0.7922 - val_loss: 0.4911
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7862 - loss: 0.4814 - val_accuracy: 0.7987 - val_loss: 0.4820
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7328 - loss: 0.5242 - val_accuracy: 0.7987 - val_loss: 0.4730
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8020 - loss: 0.4839 - val_accuracy: 0.7922 - val_loss: 0.4668
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7580 - loss: 0.4966 - val_accuracy: 0.7987 - val_loss: 0.4629
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7563 - loss: 0.4946 - val_accuracy: 0.79

In [75]:
def build_model(hp):

  model = Sequential()

  units = hp.Int('units',min_value=8,max_value=128)

  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [76]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='mydir',project_name='Units_tuner_0step')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [77]:
tuner.search(x_train_trf,y_train,epochs=5,validation_data=(x_test_trf,y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 24s


In [78]:
tuner.get_best_hyperparameters()[0].values

{'units': 49}

In [79]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [80]:
model.fit(x_train_trf,y_train,epochs=100,initial_epoch=6,batch_size=32,validation_data=(x_test_trf,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.7353 - loss: 0.5269 - val_accuracy: 0.7727 - val_loss: 0.4970
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7937 - loss: 0.4808 - val_accuracy: 0.7857 - val_loss: 0.4892
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7641 - loss: 0.4910 - val_accuracy: 0.7792 - val_loss: 0.4828
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7652 - loss: 0.4728 - val_accuracy: 0.7727 - val_loss: 0.4790
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7751 - loss: 0.4703 - val_accuracy: 0.7792 - val_loss: 0.4768
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7729 - loss: 0.4756 - val_accuracy: 0.7987 - val_loss: 0.4757
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7622 - loss: 0.4668 - val_accuracy: 0.7987 - val_loss: 0.4753
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7671 - loss: 0.4507 - val_accuracy: 0.79

In [81]:
def build_model(hp):

  model = Sequential()

  model.add(Dense(72,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

    model.add(Dense(72,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [82]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir',project_name='Layers_tuner')


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [83]:
tuner.search(x_train_trf,y_train,epochs=5,validation_data=(x_test_trf,y_test))

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.8246753215789795

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 18s


In [86]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5}

In [87]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [88]:
model.fit(x_train_trf,y_train,epochs=100,initial_epoch=6,batch_size=32,validation_data=(x_test_trf,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 66ms/step - accuracy: 0.7882 - loss: 0.4497 - val_accuracy: 0.7792 - val_loss: 0.4689
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7995 - loss: 0.4115 - val_accuracy: 0.7403 - val_loss: 0.5255
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7967 - loss: 0.4117 - val_accuracy: 0.8117 - val_loss: 0.4630
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7964 - loss: 0.4098 - val_accuracy: 0.7662 - val_loss: 0.5055
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8052 - loss: 0.4121 - val_accuracy: 0.8052 - val_loss: 0.4634
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8143 - loss: 0.3960 - val_accuracy: 0.7857 - val_loss: 0.4991
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8195 - loss: 0.3991 - val_accuracy: 0.8052 - val_loss: 0.4721
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8230 - loss: 0.3749 - val_accuracy: 0.79

### Let's do all at once

In [92]:
def build_model(hp):

  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

    if counter == 0:
      model.add(
          Dense(
              units=hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
              activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),
              input_dim=8))

    else:
      model.add(
          Dense(
              units=hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
              activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid'])))
    counter+=1

    model.add(Dense(1,activation='sigmoid'))


    model.compile(optimizer = hp.Choice('optimizer',values = ['rmsprop','adam','sgd','nadam','adamdelta']),
                  loss = 'binary_crossentropy',
                  metrics = ['accuracy'])

    return model


In [93]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir',project_name='Final_tuner')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [94]:
tuner.search(x_train_trf,y_train,epochs=5,validation_data=(x_test_trf,y_test))

Trial 3 Complete [00h 00m 01s]

Best val_accuracy So Far: 0.6883116960525513
Total elapsed time: 00h 00m 07s


In [95]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5, 'units0': 40, 'activation0': 'sigmoid', 'optimizer': 'nadam'}

In [96]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 2 variables whereas the saved optimizer has 11 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [100]:
model.fit(x_train_trf,y_train,epochs=100,initial_epoch=5,batch_size=32,validation_data=(x_test_trf,y_test))

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7660 - loss: 0.4676 - val_accuracy: 0.8052 - val_loss: 0.4559
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7972 - loss: 0.4207 - val_accuracy: 0.8117 - val_loss: 0.4551
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7694 - loss: 0.4436 - val_accuracy: 0.8052 - val_loss: 0.4557
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7679 - loss: 0.4656 - val_accuracy: 0.8052 - val_loss: 0.4553
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7744 - loss: 0.4539 - val_accuracy: 0.8052 - val_loss: 0.4553
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7695 - loss: 0.4532 - val_accuracy: 0.8052 - val_loss: 0.4551
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7979 - loss: 0.4286 - val_accuracy: 0.8052 - val_loss: 0.4545
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7797 - loss: 0.4282 - val_accuracy: 0.7987

- This file is only for practice not to extract the best model.